# Load necessary keys

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ["LANGSMITH_TRACING"] = "true"

# End to End agent

In [22]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

# Create the agent
memory = MemorySaver()
model = init_chat_model("qwen-2.5-32b", model_provider="groq")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")
print("*"*10)
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! Nice to meet you. San Francisco is a great city. Is there anything specific about SF you would like to know or discuss?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 210, 'total_tokens': 240, 'completion_time': 0.075424915, 'prompt_time': 0.013850544, 'queue_time': 0.024010331000000003, 'total_time': 0.089275459}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_7b9ecfcd5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-174c16ae-5c67-4be1-be0b-92f2644383cc-0', usage_metadata={'input_tokens': 210, 'output_tokens': 30, 'total_tokens': 240})]}}
----
**********
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_exw4', 'function': {'arguments': '{"query": "current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 

# Agent creatioln

## Tool defanition

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in kannur")
print(search_results[0])

{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Kannur', 'region': 'Kerala', 'country': 'India', 'lat': 11.8667, 'lon': 75.3653, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1739695345, 'localtime': '2025-02-16 14:12'}, 'current': {'last_updated_epoch': 1739694600, 'last_updated': '2025-02-16 14:00', 'temp_c': 30.4, 'temp_f': 86.7, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 229, 'wind_dir': 'SW', 'pressure_mb': 1008.0, 'pressure_in': 29.77, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 62, 'cloud': 2, 'feelslike_c': 34.2, 'feelslike_f': 93.6, 'windchill_c': 30.4, 'windchill_f': 86.7, 'heatindex_c': 34.2, 'heatindex_f': 93.6, 'dewpoint_c': 22.3, 'dewpoint_f': 72.2, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.7, 'gust_mph': 15.7, 'gust_kph': 25.3}}"}


In [44]:
# tools list
tools = [search]

## Define model 

In [25]:
from langchain.chat_models import init_chat_model

model = init_chat_model("qwen-2.5-32b", model_provider="groq")

In [26]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

## Combine tool with model 

In [27]:
model_with_tools = model.bind_tools(tools)

In [46]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}") # print response content
print(f"ToolCalls: {response.tool_calls}")  # print tool calls  

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [47]:
system_prompt = """
You are an expert assistant designed to answer questions by following a structured, step-by-step reasoning process. To assist you, you have access to tools that can be invoked using Python functions. Your task-solving approach should follow a sequence of 'Thought:', 'Code:', and 'Observation:' steps.

### How to Proceed:
1. Thought: At each step, explain your reasoning for answering the question. Decide whether to use a tool or not, and describe your plan.
2. Code: If a tool is needed, write the corresponding Python code in a simple format. The code block must end with `<end_code>`. Use `print()` statements to capture important intermediate results.
3. Observation: The output of the `print()` statements will appear in this field, providing information for the next step.
4. Repeat the cycle until you arrive at the final answer.

### Final Output:
At the end of the process, provide the `final_answer` to the question.

---

### Examples:

#### Example 1:
Task: "What is the weather in San Francisco?"

Thought: I need to find the current weather in San Francisco. I will use the `search` tool to look up this information.

Code:
```py
answer = search.invoke("What is the weather in San Francisco?")
print(answer)
```<end_code>

Observation: San Francisco is partly cloudy.

Final Answer: The weather in San Francisco is partly cloudy.

---

#### Example 2:
Task: "Hi, how are you?"

Thought: This is a conversational question, so I don't need to use any tools. I will respond directly.

Final Answer: I'm doing well, thank you! How about you?

---

"""

In [30]:
from langchain.schema import SystemMessage, HumanMessage  # Ensure SystemMessage is imported

response = model_with_tools.invoke([
    SystemMessage(content=system_prompt),  # System prompt
    HumanMessage(content="Hi, how are you?")
])

In [31]:
response

AIMessage(content="Final Answer: I'm here to assist you! How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 536, 'total_tokens': 554, 'completion_time': 0.045278286, 'prompt_time': 0.029357621, 'queue_time': 0.022765288, 'total_time': 0.074635907}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_7b9ecfcd5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d5c2dc1-e51a-4a12-a2d8-825bb36580f0-0', usage_metadata={'input_tokens': 536, 'output_tokens': 18, 'total_tokens': 554})

In [32]:
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Final Answer: I'm here to assist you! How can I help you today?
ToolCalls: []


In [33]:
response = model_with_tools.invoke([
    SystemMessage(content=system_prompt),  # System prompt
    HumanMessage(content="What's the weather in SF?")
])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'call_xvhm', 'type': 'tool_call'}]


In [34]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xvhm', 'function': {'arguments': '{"query": "weather in SF"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 537, 'total_tokens': 563, 'completion_time': 0.065371145, 'prompt_time': 0.02728944, 'queue_time': 0.022198402999999995, 'total_time': 0.092660585}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_7b9ecfcd5f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5d69692f-d856-4c92-b275-0f677e5832ec-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'call_xvhm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 537, 'output_tokens': 26, 'total_tokens': 563})

In [35]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ahs7', 'type': 'tool_call'}]


In [36]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [37]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='65334a66-2adc-412b-9076-204885e5d04a'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 203, 'total_tokens': 213, 'completion_time': 0.025168988, 'prompt_time': 0.011783832, 'queue_time': 0.022202860999999997, 'total_time': 0.03695282}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_7b9ecfcd5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd1b5be0-e68d-4093-85d0-931400605cb3-0', usage_metadata={'input_tokens': 203, 'output_tokens': 10, 'total_tokens': 213})]

In [38]:
response = agent_executor.invoke({"messages": [
    SystemMessage(content=system_prompt),  # System prompt
    HumanMessage(content="What's the weather in SF?")
]})

response["messages"]

[SystemMessage(content='\nYou are an expert assistant designed to answer questions by following a structured, step-by-step reasoning process. To assist you, you have access to tools that can be invoked using Python functions. Your task-solving approach should follow a sequence of \'Thought:\', \'Code:\', and \'Observation:\' steps.\n\n### How to Proceed:\n1. Thought: At each step, explain your reasoning for answering the question. Decide whether to use a tool or not, and describe your plan.\n2. Code: If a tool is needed, write the corresponding Python code in a simple format. The code block must end with `<end_code>`. Use `print()` statements to capture important intermediate results.\n3. Observation: The output of the `print()` statements will appear in this field, providing information for the next step.\n4. Repeat the cycle until you arrive at the final answer.\n\n### Final Output:\nAt the end of the process, provide the `final_answer` to the question.\n\n---\n\n### Examples:\n\n###

In [39]:
response["messages"][-1].content

"Thought: I've received the weather information for San Francisco from the API. The current temperature is 8.9°C and it's partly cloudy.\n\nFinal Answer: The current weather in San Francisco is partly cloudy with a temperature of 8.9°C."

In [40]:
response = agent_executor.invoke(
    {"messages": [
    SystemMessage(content=system_prompt),  # System prompt
    HumanMessage(content="What's the weather in SF?")
]}
)
response["messages"]



[SystemMessage(content='\nYou are an expert assistant designed to answer questions by following a structured, step-by-step reasoning process. To assist you, you have access to tools that can be invoked using Python functions. Your task-solving approach should follow a sequence of \'Thought:\', \'Code:\', and \'Observation:\' steps.\n\n### How to Proceed:\n1. Thought: At each step, explain your reasoning for answering the question. Decide whether to use a tool or not, and describe your plan.\n2. Code: If a tool is needed, write the corresponding Python code in a simple format. The code block must end with `<end_code>`. Use `print()` statements to capture important intermediate results.\n3. Observation: The output of the `print()` statements will appear in this field, providing information for the next step.\n4. Repeat the cycle until you arrive at the final answer.\n\n### Final Output:\nAt the end of the process, provide the `final_answer` to the question.\n\n---\n\n### Examples:\n\n###

In [41]:
response["messages"][1].content

"What's the weather in SF?"

In [42]:
for chunk in agent_executor.stream(
    {"messages": [
    SystemMessage(content=system_prompt),  # System prompt
    HumanMessage(content="What's the weather in delhi, india?")
]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_44sf', 'function': {'arguments': '{"query": "weather in delhi india"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 540, 'total_tokens': 568, 'completion_time': 0.071889148, 'prompt_time': 0.0295927, 'queue_time': 0.022282593000000003, 'total_time': 0.101481848}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_7b9ecfcd5f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9afcc20a-11e1-4189-8c73-10d6df208962-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in delhi india'}, 'id': 'call_44sf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 540, 'output_tokens': 28, 'total_tokens': 568})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Delhi\', \'region\': \'Delhi\

In [43]:
print(chunk['agent']['messages'][0].content)

Thought: I have received the weather information for Delhi, India. I can extract the necessary details from the data provided by the `tavily_search_results_json` function.

The current weather in Delhi, India is Mist with a temperature of 27.1°C (80.8°F). It is daytime and the humidity is 25%. The visibility is 4.5 km and the wind speed is 5.0 km/h from the west.
Final Answer: The current weather in Delhi, India is Mist with a temperature of 27.1°C (80.8°F). It is daytime and the humidity is 25%. The visibility is 4.5 km and the wind speed is 5.0 km/h from the west.
